## GluonCV on SageMaker demo

Let's first import the necessary modules. 

In [3]:
!pip install gluoncv --pre -q

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%matplotlib inline
import numpy as np
import cv2
import boto3
import json
import mxnet as mx
from gluoncv import model_zoo, data, utils
import matplotlib.pyplot as plt

### Object Detection with a pre-trained SSD model

In this demo we are going to perform inference on a GluonCV object detection model that has been deployed in Amazon SageMaker. We run this notebook on our laptop to capture image data from the webcam and to send it to the SageMaker endpoint. 

The endpoint is running the model `ssd_512_resnet50_v1_voc` which has the following structure:

In [2]:
net = model_zoo.get_model('ssd_512_resnet50_v1_voc', pretrained=False)
net

SSD(
  (features): FeatureExpander(
  
  )
  (class_predictors): HybridSequential(
    (0): ConvPredictor(
      (predictor): Conv2D(None -> 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ConvPredictor(
      (predictor): Conv2D(None -> 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): ConvPredictor(
      (predictor): Conv2D(None -> 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): ConvPredictor(
      (predictor): Conv2D(None -> 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (4): ConvPredictor(
      (predictor): Conv2D(None -> 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (5): ConvPredictor(
      (predictor): Conv2D(None -> 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (box_predictors): HybridSequential(
    (0): ConvPredictor(
      (predictor): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ConvPredictor(
      (predicto

At this step we assume that the AWS credentials have been set beforehand. If this is not the case, make sure to set the following environment variables in the commandline before starting this notebook: `AWS_ACCESS_KEY_ID` `AWS_SECRET_ACCESS_KEY` and `AWS_SESSION_TOKEN`.

In the following lines of code, we list the endpoints that have been deployed in our account.

In [3]:
runtime= boto3.client('runtime.sagemaker')
client = boto3.client('sagemaker')
client.list_endpoints()['Endpoints']

[{'EndpointName': 'gluoncv-bcbs-logo-detection',
  'EndpointArn': 'arn:aws:sagemaker:us-east-1:291790195320:endpoint/gluoncv-bcbs-logo-detection',
  'CreationTime': datetime.datetime(2019, 12, 6, 19, 0, 55, 278000, tzinfo=tzlocal()),
  'LastModifiedTime': datetime.datetime(2019, 12, 6, 19, 8, 14, 469000, tzinfo=tzlocal()),
  'EndpointStatus': 'InService'}]

### Pretrained SSD model

Next we get the endpoint that has been deployed under the name `gluoncv-pretrained-object-detection-ssd`

In [6]:
for endpoint in  client.list_endpoints()['Endpoints']:
    if endpoint["EndpointName"] == "gluoncv-bcbs-logo-detection":
        gluoncv_endpoint = endpoint

Now we capture image data from the local webcam and send it to the endpoint. The endpoint will perform inference and return bounding boxes, class ids and confidence scores.

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    img = cv2.resize(frame, (512,512))
    payload = json.dumps(img.tolist())
    response = runtime.invoke_endpoint(EndpointName=gluoncv_endpoint['EndpointName'],
                                       Body=payload)
    response_body = response['Body']
    result = json.loads(response_body.read().decode())
    [class_IDs, scores, bounding_boxes] = result
    
    bounding_boxes, scores, class_IDs =  mx.nd.array(bounding_boxes), mx.nd.array(scores), mx.nd.array(class_IDs)
    ax = utils.viz.cv_plot_bbox(img, bounding_boxes[0], scores[0], class_IDs[0], thresh=0.6, class_names=net.classes)
    
    cv2.imshow("image", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### AWS Logo detection

We have deployed the GluonCV object detection model before in SageMaker under the name `gluoncv-demo-object-detection`. Now we retrieve this endpoint.

In [3]:
runtime= boto3.client('runtime.sagemaker')
client = boto3.client('sagemaker')
client.list_endpoints()['Endpoints']

for endpoint in  client.list_endpoints()['Endpoints']:
    if endpoint["EndpointName"] == "gluoncv-bcbs-logo-detection":
        gluoncv_endpoint = endpoint
        
gluoncv_endpoint

{'EndpointName': 'gluoncv-bcbs-logo-detection',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:291790195320:endpoint/gluoncv-bcbs-logo-detection',
 'CreationTime': datetime.datetime(2019, 12, 7, 4, 15, 21, 418000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2019, 12, 7, 4, 23, 36, 915000, tzinfo=tzlocal()),
 'EndpointStatus': 'InService'}

In the following code section, we retrieve frames from the webcam and send it to the SageMaker endpoint. We first have to convert the data into JSON format.

In [ ]:
import time 
import json
cap = cv2.VideoCapture(0)
time.sleep(2)

while(True):
    ret, frame = cap.read()
    img = cv2.resize(frame, (512,512))
    payload = json.dumps(img.tolist())
    response = runtime.invoke_endpoint(EndpointName=gluoncv_endpoint['EndpointName'],
                                       Body=payload)
    response_body = response['Body']
    result = json.loads(response_body.read().decode())
    [class_IDs, scores, bounding_boxes] = result
    
    bounding_boxes, scores, class_IDs =  mx.nd.array(bounding_boxes), mx.nd.array(scores), mx.nd.array(class_IDs)
    ax = utils.viz.cv_plot_bbox(img, bounding_boxes[0], scores[0], class_IDs[0], thresh=0.6, class_names=["logo"])
    
    cv2.imshow("image", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [20]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

error: /feedstock_root/build_artefacts/opencv_1520639543262/work/opencv-3.3.0/modules/imgproc/src/color.cpp:10638: error: (-215) scn == 3 || scn == 4 in function cvtColor


In [ ]:
import math
import glob

train_images = glob.glob("images/*.jpg")

n_images = 12
cols = (int(math.sqrt(n_images)))*2
fig = plt.figure(figsize=(20,5))
imgs = []
for n, (image) in enumerate(train_images[:n_images]):
    img = cv2.imread(image)
    img = cv2.resize(img, (512,512))
    payload = json.dumps(img.tolist())
    response = runtime.invoke_endpoint(EndpointName=gluoncv_endpoint['EndpointName'],
                                       Body=payload)
    response_body = response['Body']
    result = json.loads(response_body.read().decode())
    [class_IDs, scores, bounding_boxes] = result
   
    bounding_boxes, scores, class_IDs =  mx.nd.array(bounding_boxes), mx.nd.array(scores), mx.nd.array(class_IDs)
    ax = utils.viz.cv_plot_bbox(img, bounding_boxes[0], scores[0], class_IDs[0], thresh=0.6, class_names=["logo"])
    img = cv2.resize(img, (700, 700))
    #plt.imshow(img)
    imgs.append(img)   
    
w=10
h=10
fig=plt.figure(figsize=(60, 60))
columns = 1
rows = 11
for i in range(1, columns*rows +1):
    #img = np.random.randint(10, size=(h,w))
    fig.add_subplot(rows, columns, i)
    plt.imshow(imgs[i])
plt.show()

In [23]:
import base64
from io import BytesIO
from PIL import Image
import cv2
from utils import show_webcam

def get_annotated_image(input_image_b64):
    prefix, input_image_b64 = input_image_b64.split(',')
    input_image_binary = BytesIO(base64.b64decode(input_image_b64))
    input_image_np = np.asarray(Image.open(input_image_binary))
    input_image_np, _ = mx.image.center_crop(mx.nd.array(input_image_np), (512,512))
    _, input_image_loaded = gcv.data.transforms.presets.yolo.transform_test(input_image_np, short=384)
    
    payload = json.dumps(input_image_loaded.tolist())
    response = runtime.invoke_endpoint(EndpointName=gluoncv_endpoint['EndpointName'],
                                       Body=payload)
    response_body = response['Body']
    result = json.loads(response_body.read().decode())
    [class_IDs, scores, bounding_boxes] = result
    bounding_boxes, scores, class_IDs =  mx.nd.array(bounding_boxes), mx.nd.array(scores), mx.nd.array(class_IDs)
    output_image_np = gcv.utils.viz.cv_plot_bbox(input_image_loaded, bounding_boxes[0], scores[0], class_IDs[0], thresh=0.6, class_names=["logo"])
    
    #output = deployed_model.predict(input_image_loaded)
    #cid = np.array(output['cid'])
    #scores = np.array(output['score'])
    #bbox = np.array(output['bbox'])
    #output_image_np = gcv.utils.viz.cv_plot_bbox(input_image_loaded, bbox, scores, cid, class_names=classes)
    output_image_PIL = Image.fromarray(output_image_np)
    output_buffer = BytesIO()
    output_image_PIL.save(output_buffer, format="JPEG")
    output_image_b64 = 'data:image/jpeg;base64,'+base64.b64encode(output_buffer.getvalue()).decode("utf-8")
    print(output_image_b64)

show_webcam()
#display()

In [8]:
img = cv2.resize(frame, (512,512))
    payload = json.dumps(img.tolist())
    response = runtime.invoke_endpoint(EndpointName=gluoncv_endpoint['EndpointName'],
                                       Body=payload)
    response_body = response['Body']
    result = json.loads(response_body.read().decode())
    [class_IDs, scores, bounding_boxes] = result
    
    bounding_boxes, scores, class_IDs =  mx.nd.array(bounding_boxes), mx.nd.array(scores), mx.nd.array(class_IDs)
    ax = utils.viz.cv_plot_bbox(img, bounding_boxes[0], scores[0], class_IDs[0], thresh=0.6, class_names=["logo"])
    
    cv2.imshow("image", img)

<Figure size 1440x360 with 0 Axes>